In [13]:
from skyfield import api
from skyfield.api import *
from pytz import timezone
import datetime as dt
from skyfield.searchlib import find_discrete
from skyfield import almanac


eph = api.load('de440s.bsp')
earth, sun, moon = eph['earth'], eph['sun'], eph['moon']

lok = earth + wgs84.latlon(5.41*N, 100.18962*E)

def lokasi(latitud, longitud):
    zone = timezone('Asia/Kuala_Lumpur')
    lokasi_tempatan = wgs84.latlon(latitud*N, longitud*E)
    return  lokasi_tempatan, latitud

def waktu(tahun, bulan, hari):
    zone = timezone('Asia/Kuala_Lumpur')
    now = zone.localize(dt.datetime(tahun, bulan, hari))
    midnight = now.replace(hour = 0, minute = 0, second = 0, microsecond = 0)
    next_midnight = midnight + dt.timedelta(days =1)
    
    ts = load.timescale()
    t0 = ts.from_datetime(midnight)
    t1 = ts.from_datetime(next_midnight)
    
    return t0, t1

def altitud_matahari_declination(t, lokasi):
    earth = eph['earth']
    lok = earth + lokasi
    from_location_on_earth = lok.at(t)
    earth_observe_sun = from_location_on_earth.observe(sun)
    apparent_position = earth_observe_sun.apparent()
    al, az, jar = apparent_position.altaz()
    ra, dec, jar = apparent_position.radec()
    alti = al.degrees
    deklinasi = dec.degrees
    return alti, deklinasi

def waktu_zohor(lokasi, t0, t1):
    eph = api.load('de440s.bsp')
    zone = timezone('Asia/Kuala_Lumpur')
    cari_transit = almanac.meridian_transits(eph, eph['Sun'], lokasi)
    waktu, posisi = almanac.find_discrete(t0, t1, cari_transit)
    
    zuhr = waktu[posisi == 1]
    wak_zohor = zuhr[0]
    zohor = str(wak_zohor.astimezone(zone) + dt.timedelta(minutes =1, seconds = 4))[11:19]
    return zohor, wak_zohor

    
def waktu_subuh_syuruk_maghrib_isyak(lokasi, t0, t1):
    eph = api.load('de440s.bsp')
    zone = timezone('Asia/Kuala_Lumpur')
    fajar_syafak = almanac.dark_twilight_day(eph, lokasi)
    waktu, posisi = almanac.find_discrete(t0, t1, fajar_syafak)
    
    x = 0
    for t, e in zip(waktu, posisi):
        waktu_solat = str(t.astimezone(zone))[11:19]
        if x == 0:
            subuh = waktu_solat
        elif x == 3:
            syuruk = waktu_solat
        elif x == 4:
            maghrib = waktu_solat
        elif x == 7:
            isyak = waktu_solat
        x += 1
    
    
    return subuh, syuruk, maghrib, isyak




In [14]:
from skyfield.api import *
from pytz import timezone
import datetime as dt
from skyfield import almanac
from skyfield import *

def lokasi2(latitud, longitud):
    zone = timezone('Asia/Kuala_Lumpur')
    lokasi_tempatan = wgs84.latlon(latitud*N, longitud*E)
    return  lokasi_tempatan
def waktu_maghrib(lokasi, tahun, bulan, hari):
    zone = timezone('Asia/Kuala_Lumpur')
    set_masa = zone.localize(dt.datetime(tahun, bulan, hari))
    set_awal_hari = set_masa.replace(hour= 0, minute= 0, second= 0, microsecond= 0)
    set_akhir_hari = set_awal_hari + dt.timedelta(days = 1)
    
    ts = load.timescale()
    awal_hari = ts.from_datetime(set_awal_hari)
    akhir_hari = ts.from_datetime(set_akhir_hari)
    
    eph = load('de440s.bsp')
    terbenam_matahari = almanac.sunrise_sunset(eph, lokasi)
    
    waktu_maghrib_datetime, terbenam = almanac.find_discrete(awal_hari, akhir_hari, terbenam_matahari)
    maghrib_skyfield = waktu_maghrib_datetime[terbenam == 0]
    
    maghrib_1 = maghrib_skyfield[0]
    waktu_maghrib1 = str(maghrib_1.astimezone(zone))[11:19]
    tarikh = str(maghrib_1.astimezone(zone))[:11]
    
    return maghrib_1, waktu_maghrib1, tarikh

def altitud_jarak_lengkung_bulan(waktu, lokasi_tempatan):
    planet = load('de440s.bsp')
    bumi, bulan, matahari = planet['earth'], planet['moon'], planet['sun']
    ephemeris = api.load('de440s.bsp')
    lok = bumi + lokasi_tempatan
    lokasi_bulan = lok.at(waktu[0]).observe(bulan)
    lokasi_matahari = lok.at(waktu[0]).observe(matahari)
    jarak_lengkung = lokasi_matahari.separation_from(lokasi_bulan)
    jarak_lengkung_darjah = '%.4f' %jarak_lengkung.degrees
    altitud_bulan, az_bulan, jarak_bulan = lokasi_bulan.apparent().altaz()
    altitud_bulan_darjah = '%.4f' %altitud_bulan.degrees
    
    return jarak_lengkung_darjah, altitud_bulan_darjah
def imkanur_rukyah__syarat_643 (lokasi, tahun_masihi, hijri_hari_pertama, hijri_bulan_pertama):
    hijri_hari = hijri_hari_pertama
    hijri_bulan = hijri_bulan_pertama
    list_hijri_hari = [hijri_hari]
    list_hijri_bulan = [hijri_bulan]
    tarikh_masihi_list = []
    for i in range(1,13):
        for f in range (1,32):
            if (i in [2,4,6,9,11] and f >30) or (i == 2 and f > 28):
                continue 
            wak_magh = waktu_maghrib(lokasi, tahun_masihi, i, f)
            tarikh_masihi_list.append(wak_magh[2])
            alt_jar_bulan = altitud_jarak_lengkung_bulan(wak_magh, lokasi)
            jarak_lengkung = float(alt_jar_bulan[0])
            altitud_bulan = float(alt_jar_bulan[1])
            if hijri_hari < 29 and hijri_bulan < 13:
                hijri_hari += 1
                list_hijri_hari.append(hijri_hari)
                list_hijri_bulan.append(hijri_bulan)
            elif hijri_hari == 30 and hijri_bulan <12:
                hijri_bulan += 1
                hijri_hari = 1
                list_hijri_hari.append(hijri_hari)
                list_hijri_bulan.append(hijri_bulan)
            elif hijri_hari == 30 and hijri_bulan == 12:
                hijri_hari = 1
                hijri_bulan = 1
                list_hijri_hari.append(hijri_hari)
                list_hijri_bulan.append(hijri_bulan)
            elif hijri_hari == 29 and hijri_bulan < 13:
                if jarak_lengkung < 6.4:
                    hijri_hari += 1
                    list_hijri_hari.append(hijri_hari)
                    list_hijri_bulan.append(hijri_bulan)
                elif jarak_lengkung > 6.4 and altitud_bulan < 3:
                    hijri_hari += 1
                    list_hijri_hari.append(hijri_hari)
                    list_hijri_bulan.append(hijri_bulan)
                else:
                    if hijri_bulan == 12:
                        hijri_hari = 1
                        hijri_bulan = 1
                        list_hijri_hari.append(hijri_hari)
                        list_hijri_bulan.append(hijri_bulan)
                    else:
                        hijri_hari = 1
                        hijri_bulan += 1
                        list_hijri_hari.append(hijri_hari)
                        list_hijri_bulan.append(hijri_bulan)
    return list_hijri_hari, list_hijri_bulan, tarikh_masihi_list

In [6]:
def altitud_matahari_ketika_asar(waktu_transit, lokasi):
    earth = eph['earth']
    lok = earth + lokasi[0]
    from_location_on_earth = lok.at(waktu_transit)
    earth_observe_sun = from_location_on_earth.observe(sun)
    apparent_position = earth_observe_sun.apparent()
    ra, dec, jar = apparent_position.radec()
    dek = dec.degrees
    theta_1 = float((lokasi[1] - dek)*pi/180)
    theta_2 = float(acot(tan(theta_1) + 1))
    altitud_asar = float(degrees(theta_2))
    
    return altitud_asar

def waktu2(tahun, bulan, hari):
    zone = timezone('Asia/Kuala_Lumpur')
    now = zone.localize(dt.datetime(tahun, bulan, hari))
    midnight = now.replace(hour = 12, minute = 0, second = 0, microsecond = 0)
    next_midnight = midnight + dt.timedelta(hours = 6)
    
    ts = load.timescale()
    t3 = ts.from_datetime(midnight)
    t4 = ts.from_datetime(next_midnight)
    
    return t3, t4



In [7]:
def takwim_solat(tahun, latitud, longitud):
    takwim_subuh = []
    takwim_syuruk = []
    takwim_zohor = []
    takwim_asar = []
    takwim_maghrib = []
    takwim_isyak = []
    for i in range(1,13):
            for f in range (1,32):
                if (i in [2,4,6,9,11] and f >30) or (i == 2 and f > 28):
                    continue
                wak = waktu(tahun, i, f)
                tempat = lokasi(latitud, longitud)
                zohor = waktu_zohor(tempat[0],wak[0], wak[1])
                w = waktu_zohor(tempat[0], wak[0],wak[1])
                altitud_matahari_ketika_asar(w[1], tempat)
                subuh = waktu_subuh_syuruk_maghrib_isyak(tempat[0],wak[0], wak[1])[0]
                syuruk = waktu_subuh_syuruk_maghrib_isyak(tempat[0],wak[0], wak[1])[1]
                maghrib = waktu_subuh_syuruk_maghrib_isyak(tempat[0],wak[0], wak[1])[2]
                isyak = waktu_subuh_syuruk_maghrib_isyak(tempat[0],wak[0], wak[1])[3]
                takwim_subuh.append(subuh)
                takwim_syuruk.append(syuruk)
                takwim_maghrib.append(maghrib)
                takwim_isyak.append(isyak)
                takwim_zohor.append(zohor[0])
    return takwim_subuh, takwim_syuruk, takwim_zohor, takwim_maghrib, takwim_isyak

In [8]:
from mpmath import *
takwim_asar = []
zone = timezone('Asia/Kuala_Lumpur')
for i in range(1,13):
    for f in range (1,32):
                if (i in [2,4,6,9,11] and f >30) or (i == 2 and f > 28):
                    continue
                wak = waktu2(2021, i, f)
                y = lokasi(5.44, 100.2)
                w = waktu_zohor(y[0], wak[0],wak[1])
                z = altitud_matahari_ketika_asar(w[1], y)
                def algoritma_asar(t):
                    earth = eph['earth']
                    lok = earth + y[0]
                    from_location_on_earth = lok.at(t)
                    earth_observe_sun = from_location_on_earth.observe(sun)
                    apparent_position = earth_observe_sun.apparent()
                    al, az, jar = apparent_position.altaz()
                    alti = al.degrees
                    x = abs(alti-z)
                    return x < 0.01

                algoritma_asar.step_days = 1/86400

                def list_waktu_asar(waktu):
                    asar, nilai = find_discrete(waktu[0], waktu[1], algoritma_asar)

                    waktu_asar = str(asar[0].astimezone(zone))[11:19]
                    return waktu_asar

                r = list_waktu_asar(wak)
                takwim_asar.append(r)

In [15]:
import pandas as pd
tak = takwim_solat(2021, 5.44, 100.2)
lokasi_anda = lokasi2(5.44, 100.2)
takwim = imkanur_rukyah__syarat_643(lokasi_anda, 2021, 17, 5)
takwim_waktu_solat = pd.DataFrame(list(zip(takwim[0], takwim[1], takwim[2], tak[0], tak[1], tak[2],takwim_asar, tak[3], tak[4])),
                                  columns=['hari', 'bulan', 'masihi','subuh','syuruk', 'zohor','asar', 'maghrib', 'isyak'])
takwim_waktu_solat

,hari,bulan,masihi,subuh,syuruk,zohor,asar,maghrib,isyak
0,17,5,2021-01-01,06:13:33,07:28:15,13:23:48,16:45:20,19:17:14,20:31:55
1,18,5,2021-01-02,06:14:01,07:28:41,13:24:16,16:45:49,19:17:44,20:32:22
2,19,5,2021-01-03,06:14:30,07:29:07,13:24:44,16:46:17,19:18:14,20:32:49
3,20,5,2021-01-04,06:14:58,07:29:32,13:25:11,16:46:45,19:18:44,20:33:16
4,21,5,2021-01-05,06:15:26,07:29:56,13:25:38,16:47:13,19:19:13,20:33:42
...,...,...,...,...,...,...,...,...,...
360,22,5,2021-12-27,06:10:56,07:25:50,13:21:15,16:42:44,19:14:32,20:29:25
361,23,5,2021-12-28,06:11:26,07:26:19,13:21:44,16:43:14,19:15:03,20:29:54
362,24,5,2021-12-29,06:11:56,07:26:47,13:22:14,16:43:44,19:15:33,20:30:23
363,25,5,2021-12-30,06:12:26,07:27:14,13:22:43,16:44:13,19:16:04,20:30:51


In [16]:
takwim_waktu_solat.to_csv('takwim1.csv')